In [1]:
import tensorflow as tf
import os 
import cPickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print os.listdir(CIFAR_DIR)

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = cPickle.load(f)
        return data['data'], data['labels']
    
# tensorflow.Dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print self._data.shape
        print self._labels.shape
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0, 1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print batch_data
# print batch_labels

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
# None表示输入的样本数量的不确定的
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0, 5, 6, 3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32 * 32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# 神经元图，feature_map，输出图像
conv1 = tf.layers.conv2d(x_image,
                        32, # 卷积核的数量
                        (3,3), # 卷积核的大小
                        padding='same', # same是做padding使输出的大小和输入的大小一致，valid是不做padding
                        activation = tf.nn.relu,
                        name = 'conv1')
# 16 * 16
pooling1 = tf.layers.max_pooling2d(conv1,
                                  (2,2), # 池化核大小
                                  (2,2), # 池化核步长
                                  name='pool1')

conv2 = tf.layers.conv2d(pooling1,
                        32, # 卷积核的数量
                        (3,3), # 卷积核的大小
                        padding='same', # same是做padding使输出的大小和输入的大小一致，valid是不做padding
                        activation = tf.nn.relu,
                        name = 'conv2')
# 8 * 8
pooling2 = tf.layers.max_pooling2d(conv2,
                                  (2,2), # 池化核大小
                                  (2,2), # 池化核步长
                                  name='pool2')


conv3 = tf.layers.conv2d(pooling2,
                        32, # 卷积核的数量
                        (3,3), # 卷积核的大小
                        padding='same', # same是做padding使输出的大小和输入的大小一致，valid是不做padding
                        activation = tf.nn.relu,
                        name = 'conv3')
# 4 * 4 * 32
pooling3 = tf.layers.max_pooling2d(conv3,
                                  (2,2), # 池化核大小
                                  (2,2), # 池化核步长
                                  name='pool3')

# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

# 平方差损失函数
"""
# course: 1 + e^x
# api: e^x / sum(e^x)
# [[0.01, 0.9, ..., 0.03], []]
p_y = tf.nn.softmax(y_)
# 5 -> [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)
# square平方，mean均值
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""



# 交叉熵损失函数
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_



"""
# [None, 1]
p_y_1 = tf.nn.sigmoid(y_)
# [None, 1]
y_reshaped = tf.reshape(y, (-1, 1))
y_reshaped_float = tf.cast(y_reshaped, tf.float32)

# square平方，mean均值
loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))
"""

# indices
predict = tf.argmax(y_, 1)
# [1, 2, 5, 1, 3, 0, 7, 0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    # AdamOptimizer梯度下降
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={
                x: batch_data, 
                y: batch_labels
            })
        if (i+1) % 500 == 0:
            print '[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, acc_val)
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy], 
                    feed_dict={
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print '[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc)

[Train] Step: 500, loss: 1.83988, acc: 0.35000
[Train] Step: 1000, loss: 1.24343, acc: 0.55000
[Train] Step: 1500, loss: 1.33171, acc: 0.40000
[Train] Step: 2000, loss: 1.12933, acc: 0.50000
[Train] Step: 2500, loss: 1.09694, acc: 0.70000
[Train] Step: 3000, loss: 1.23935, acc: 0.55000
[Train] Step: 3500, loss: 1.13903, acc: 0.70000
[Train] Step: 4000, loss: 1.11591, acc: 0.55000
[Train] Step: 4500, loss: 0.77969, acc: 0.70000
[Train] Step: 5000, loss: 1.21630, acc: 0.50000
(10000, 3072)
(10000,)
[Test ] Step: 5000, acc: 0.67200
[Train] Step: 5500, loss: 0.78457, acc: 0.70000
[Train] Step: 6000, loss: 1.14441, acc: 0.65000
[Train] Step: 6500, loss: 0.75299, acc: 0.65000
[Train] Step: 7000, loss: 0.75635, acc: 0.75000
[Train] Step: 7500, loss: 0.84783, acc: 0.80000
[Train] Step: 8000, loss: 0.81000, acc: 0.65000
[Train] Step: 8500, loss: 0.97110, acc: 0.65000
[Train] Step: 9000, loss: 0.75807, acc: 0.70000
[Train] Step: 9500, loss: 0.75448, acc: 0.65000
[Train] Step: 10000, loss: 0.5225